In [1]:
# Import list

import numpy as np
import re

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelBinarizer
from torch.optim.lr_scheduler import ReduceLROnPlateau  # Import ReduceLROnPlateau
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Read Data 

In [2]:
# Different Editors paths
PATH_A = "data\sherlock-holm.es_stories_plain-text_advs.txt"
PATH_E = ""
PATH_G = "0. Projects/3/Project-III/data/sherlock-holm.es_stories_plain-text_advs.txt"
PATH_J = ""
PATH_M = "Project-III/data/sherlock-holm.es_stories_plain-text_advs.txt"

PATHS = [PATH_A, PATH_E, PATH_G, PATH_J, PATH_M]

text = ""

for path in PATHS:
    try:
        # Read the text file
        with open(path, 'r', encoding='utf-8') as file:
            text = file.read()
            print(path)
    except:
        continue
    else:
        break



print(text[:200])

Project-III/data/sherlock-holm.es_stories_plain-text_advs.txt




                        THE ADVENTURES OF SHERLOCK HOLMES

                               Arthur Conan Doyle



                                Table of contents

               A Scandal in Bohem


## Data Preprocess

### Divide the set

In [3]:
# Divide with regular expressions
DIVIDERS_ORIGINAL = "\n"
DIVIDERS_ALL = "[,.!?:;\"]|\n\n|--| and | that | which "
DIVIDERS_MIN = "[.!]|\n\n"
DIVIDERS_BAL = "[,.!?]|\n\n|--"
CLEAR_COVER = False

text_try = text.lower()

if CLEAR_COVER:
    # Delete cover of book and extra information
    text_try = text[980:-550]
    

# Split following the dividers given
text_try = re.split(DIVIDERS_ORIGINAL, text_try)

# Delete all the new line comments 
text_try = [el.replace('\n', '') for el in text_try]

text_try[:10]

['',
 '',
 '',
 '',
 '                        the adventures of sherlock holmes',
 '',
 '                               arthur conan doyle',
 '',
 '',
 '']

### Tokenization

In [4]:
# Create Tokenizer object in python
CLEAR_COVER
tokens = word_tokenize(text)
vocabulary = set(tokens)
total_words = len(vocabulary) + 1

word_to_idx = {word:idx for idx, word in enumerate(vocabulary)}

print(f"total_words: {total_words}")
print(f"Índice de palabras: {word_to_idx}")

total_words: 8965
Índice de palabras: {'stable': 0, 'lemon': 1, 'regained': 2, 'Dundas': 3, 'introducing': 4, 'pursuers': 5, 'vulnerable': 6, 'lanterns': 7, "'Oh": 8, 'soldiers': 9, 'sank': 10, 'stopping': 11, 'escaping': 12, 'allusion': 13, 'incalculable': 14, 'thoughtless': 15, 'implicating': 16, 'freed': 17, 'dinner': 18, 'Probably': 19, 'entrance': 20, 'fiercely': 21, 'bored': 22, 'scales': 23, 'Showing': 24, 'finish': 25, 'dived': 26, '£120': 27, 'identify': 28, 'sight': 29, 'red': 30, 'understanding': 31, 'bring': 32, 'taller': 33, 'instituted': 34, 'wanted': 35, 'refuse': 36, 'feasible': 37, 'fears': 38, 'embarrassed': 39, 'defray': 40, 'staccato': 41, 'foppishness': 42, 'lightning': 43, 'bones': 44, 'crinkled': 45, 'trunks': 46, 'Gustave': 47, "lady's": 48, 'filling': 49, 'frighten': 50, 'wearer': 51, 'Honoria': 52, 'mole': 53, 'conscience': 54, 'quill': 55, 'custom': 56, 'country-houses': 57, 'aid': 58, '4d': 59, 'masses': 60, 'brain-fever': 61, 'driving': 62, 'destruction': 6

In [5]:
# Create input-output sequences
input_sequences = []
for line in text_try:
    line_list = line.rstrip(",.;:").split(' ')

    # Tokenize each sentence
    token_list = []
    for char in line_list:
        if char in word_to_idx.keys():
            token_list.append(word_to_idx[char])

    # Divide the different sentences in n-grams
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

input_sequences

[[1320, 8167],
 [1320, 8167, 8274],
 [3390, 8274],
 [3390, 8274, 5651],
 [1714, 5077],
 [1714, 5077, 7549],
 [1320, 423],
 [1320, 423, 200],
 [1714, 3229],
 [1714, 3229, 8274],
 [1714, 3229, 8274, 5636],
 [1320, 1487],
 [1320, 5336],
 [1320, 5336, 6704],
 [1320, 5336, 6704, 2836],
 [1320, 4580],
 [1320, 4580, 5177],
 [1320, 4580, 5177, 1320],
 [1320, 4580, 5177, 1320, 4324],
 [1320, 4580, 5177, 1320, 4324, 933],
 [1320, 4169],
 [1320, 4169, 8274],
 [1320, 4169, 8274, 1320],
 [1320, 4169, 8274, 1320, 4216],
 [1320, 4169, 8274, 1320, 4216, 1230],
 [1320, 4169],
 [1320, 4169, 8274],
 [1320, 4169, 8274, 1320],
 [1320, 4169, 8274, 1320, 4236],
 [1320, 4169, 8274, 1320, 4236, 3464],
 [1320, 4169],
 [1320, 4169, 8274],
 [1320, 4169, 8274, 1320],
 [1320, 4169, 8274, 1320, 3302],
 [1320, 4169],
 [1320, 4169, 8274],
 [1320, 4169, 8274, 1320],
 [1320, 4169, 8274, 1320, 6382],
 [1320, 4169, 8274, 1320, 6382, 2961],
 [1320, 4169],
 [1320, 4169, 8274],
 [1320, 4169, 8274, 1320],
 [1320, 4169, 8274, 

### Padding

In [6]:
# Get the max value to add padding to other entries

average = 0
for seq in input_sequences:
    average += len(seq) 

max_sequence_len, value = max([(len(seq), seq) for seq in input_sequences])
input_seq_pad = np.array([np.pad(seq, (max_sequence_len - len(seq), 0), mode='constant') for seq in input_sequences])

print (f"average = {average / len(input_sequences)}")
print (f"Max seq length = {max_sequence_len}")
input_seq_pad

average = 6.213641593749164
Max seq length = 16


array([[   0,    0,    0, ...,    0, 1320, 8167],
       [   0,    0,    0, ..., 1320, 8167, 8274],
       [   0,    0,    0, ...,    0, 3390, 8274],
       ...,
       [   0,    0,    0, ..., 3801, 6174, 1320],
       [   0,    0,    0, ..., 6174, 1320, 6447],
       [   0,    0,    0, ..., 1320, 6447, 6162]])

## Model Train

### X and Y separation

In [7]:
# Split the sequences into input (X) and output (y)
X = input_seq_pad[:, :-1]
y = input_seq_pad[:, -1]

# Convert output to one-hot encoded vectors
y = np.array(torch.nn.functional.one_hot(torch.tensor(y), num_classes=total_words))

print (X)
y

[[   0    0    0 ...    0    0 1320]
 [   0    0    0 ...    0 1320 8167]
 [   0    0    0 ...    0    0 3390]
 ...
 [   0    0    0 ... 1440 3801 6174]
 [   0    0    0 ... 3801 6174 1320]
 [   0    0    0 ... 6174 1320 6447]]


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [8]:
# Create a custom Dataset class
class TextDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.long)
        self.y = torch.tensor(y, dtype=torch.float)
        

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

dataset = TextDataset(X, y)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

## MODELS

### Multiple LSTM layers

In [12]:
# Define the model
class NextWordPredictor(nn.Module):
  def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim, dropout=0.2):
    super(NextWordPredictor, self).__init__()
    self.embedding = nn.Embedding(vocab_size, embed_dim)
    self.dropout_embed = nn.Dropout(dropout)  # Add dropout after embedding
    self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers=2, dropout=dropout, bidirectional=True, batch_first=True)  # Use Bidirectional LSTM with multiple layers
    self.fc = nn.Linear(hidden_dim * 2, output_dim)  # Adjust output size for Bidirectional LSTM

  def forward(self, sequences):
    embedded = self.embedding(sequences)
    embedded = self.dropout_embed(embedded)
    lstm_out, _ = self.lstm(embedded)
    last_hidden = lstm_out[:, -1, :]  # Select last hidden state from the sequence
    logits = self.fc(last_hidden)
    return logits



### Simple LSTM

In [9]:
# Train the model
class NextWordPredictor(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim):
        super(NextWordPredictor, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        x = x[:, -1, :]
        x = self.fc(x)
        return x

### Model creation


In [10]:
model = NextWordPredictor(vocab_size = total_words, 
embed_dim = 100, hidden_dim = 150, output_dim = total_words)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


## TRAINING

In [13]:
SAVE_MODEL = True  #If set to True, saves a torch_model.pt 
SAVED_MODEL_NAME = '[name].pt'

epochs = 200
patience = 3  # Number of epochs to wait for improvement
current_patience = patience
best_loss = float('inf')  # Initialize best loss to a very high value
better_model = model
for epoch in range(epochs):
  # Training loop
  for i, (inputs, labels) in enumerate(dataloader):
    outputs = model(inputs)
    loss = criterion(outputs, labels.argmax(dim=1))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  # Early stopping
  if loss.item() < best_loss:  # Compare current training loss with best loss
    best_loss = loss.item()
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()} (Improved)')
    better_model = model
    current_patience = patience  # Restart patience
  else:
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')
    current_patience -= 1  # Decrement patience counter on no improvement

  # Stop training if patience is 0
  if current_patience == 0:
    print('Early stopping triggered!')
    break



#Saving model to .pt
if SAVE_MODEL == True:
  torch.save(better_model, SAVED_MODEL_NAME)



Epoch 1/200, Loss: 4.477021217346191 (Improved)
Epoch 2/200, Loss: 3.824334144592285 (Improved)
Epoch 3/200, Loss: 3.5328009128570557 (Improved)
Epoch 4/200, Loss: 3.440668821334839 (Improved)
Epoch 5/200, Loss: 3.1337404251098633 (Improved)
Epoch 6/200, Loss: 3.162240982055664
Epoch 7/200, Loss: 3.144766330718994
Epoch 8/200, Loss: 2.457170248031616 (Improved)
Epoch 9/200, Loss: 2.646491527557373
Epoch 10/200, Loss: 2.4057536125183105 (Improved)
Epoch 11/200, Loss: 2.299654960632324 (Improved)
Epoch 12/200, Loss: 1.9355229139328003 (Improved)
Epoch 13/200, Loss: 2.2850582599639893
Epoch 14/200, Loss: 1.9784753322601318
Epoch 15/200, Loss: 1.5945571660995483 (Improved)
Epoch 16/200, Loss: 2.205716133117676
Epoch 17/200, Loss: 1.5091036558151245 (Improved)
Epoch 18/200, Loss: 1.606704592704773
Epoch 19/200, Loss: 1.852048635482788
Epoch 20/200, Loss: 1.3881460428237915 (Improved)
Epoch 21/200, Loss: 1.7224222421646118
Epoch 22/200, Loss: 1.1004705429077148 (Improved)
Epoch 23/200, Loss:

### Load model

In [23]:
# If model loaded do not run training
MODEL_PATH = 'best_model.pt'

better_model = torch.load(MODEL_PATH) 



## PREDICTION

In [28]:
# Initial text to predict
seed_text = "I am"
next_words = 10

# Index to word
idx_to_word = {idx: word for word, idx in word_to_idx.items()}

# Generate the n next words
better_model.eval()  # Set the model to evaluation
for _ in range(next_words):
    tokens = word_tokenize(seed_text)
    token_list = [word_to_idx[word] for word in tokens if word in word_to_idx]
    token_list = np.pad(token_list, (max_sequence_len - len(token_list), 0), mode='constant')
    token_list = torch.tensor(token_list[-max_sequence_len:], dtype=torch.long).unsqueeze(0)

    with torch.no_grad():
        predicted = better_model(token_list).argmax(dim=1).item()

    output_word = idx_to_word[predicted]
    seed_text += " " + output_word


print(seed_text)

I am runing with the so that it may have not been but the
